In [1]:
import pickle
import torch
import numpy as np
import IPython.display as ipd

In [2]:
from math import ceil
from librosa.filters import mel
from librosa.util import normalize
from melgan.interface import *

In [3]:
from factory.AutoVC import AutoVC
from factory.MetaConv import MetaConv

In [4]:
LEN_CROP = 176
DIM_NECK = 44
DIM_EMB = 256
D_MODEL = 80
FREQ = 22
BATCH_SIZE = 2
DEVICE = 'cpu'
ROOT = 'train_data'

In [5]:
conv = MetaConv(44,256,512,22).to(DEVICE) 
conv.load_state_dict(torch.load("model/autovc_pytorch_meta_conv_final_en.pt", map_location=DEVICE))

<All keys matched successfully>

In [6]:
vocoder = MelVocoder(model_name='model/static/multi_speaker')

In [7]:
metadata = pickle.load(open(f'{ROOT}/train.pkl', "rb"))

In [48]:
source = 0
target = 11
sound = 3

In [49]:
metadata[source][0],metadata[target][0]

('p225', 'p237')

In [50]:
emb_org = np.expand_dims(metadata[source][1],axis=0)
emb_org.shape

(1, 256)

In [51]:
p_ = metadata[source][sound].replace("\\", "/")
uttr_org = np.load(f"{ROOT}/{p_}")[50:226]
uttr_org  = np.expand_dims(uttr_org ,axis=0)
uttr_org.shape

(1, 176, 80)

In [52]:
emb_trg =  np.expand_dims(metadata[target][1],axis=0)
emb_trg.shape

(1, 256)

## 原始聲音

In [53]:
audio_s = vocoder.inverse(torch.from_numpy(np.transpose(uttr_org,(0,2,1))))
ipd.Audio(audio_s,rate = 22050)

## 目標聲音

In [54]:
uttr_target = np.expand_dims(np.load(f"{ROOT}//{metadata[target][sound]}").T,axis=0)
audio_t = vocoder.inverse(torch.from_numpy(uttr_target))
ipd.Audio(audio_t,rate = 22050)

In [55]:
_,x_conv,_ = conv(torch.from_numpy(uttr_org).to(DEVICE),torch.from_numpy(emb_org).to(DEVICE),torch.from_numpy(emb_trg).to(DEVICE))
mel_conv = x_conv[0,:,:]

In [56]:
conv_audio = vocoder.inverse(mel_conv.transpose(1,2))

## 轉換聲音

In [57]:
ipd.Audio(conv_audio,rate = 22050)